Step 1 - cleaning data from CSV file 

1.1 - deducting duplicte emails 

In [39]:

import pandas as pd

# Load the CSV
file_path = 'Data.csv'
# reading th csv file using pandas
df = pd.read_csv(file_path)

# Identify masked emails (those that contain "*****")
masked = df['email'].str.contains(r'\*{5}', na=False)
unmasked_emails = df[~masked]

# Count duplicates
total_unmasked = len(unmasked_emails)
unique_unmasked = unmasked_emails["email"].nunique()
duplicates_dropped = total_unmasked - unique_unmasked

print(f"Duplicate email entries dropped: {duplicates_dropped}")


Duplicate email entries dropped: 0


So,  no duplicate emails obtain as i took " ***** " as unique email.

1.2 - Replaceing has_joined_events from YES/NO to True/ False

In [40]:
df["has_joined_event"] = df["has_joined_event"].str.strip().str.lower().map({
    "yes": True,
    "no": False
})


# Print the only has joined event column with total number of rows
print(f"Total rows in 'has_joined_event' column: {len(df['has_joined_event'])}")
print(df["has_joined_event"].head(11))   



Total rows in 'has_joined_event' column: 648
0     False
1     False
2     False
3     False
4      True
5     False
6      True
7      True
8      True
9      True
10     True
Name: has_joined_event, dtype: bool


So, data gas been manpluted

1.3 dentify and flag rows where:
 - LinkedIn profile is missing or incomplete 
 - The job title is blank 

flaging linkdin profiles in 3 types
- valid 
- missing 
- incorrect 


In [41]:
# Clean and analyze the LinkedIn column
linkedin_col = 'What is your LinkedIn profile?'

def check_linkedin_status(link):
    if pd.isna(link) or str(link).strip() == '':
        return 'Missing'
    link = str(link).strip().lower()
    if 'linkedin.com/in/' in link:
        return 'Valid'
    return 'Incomplete'

# Apply the function to classify each LinkedIn entry
df['LinkedIn Status'] = df[linkedin_col].apply(check_linkedin_status)

# Display the counts for each status and show a preview of the flagged data
status_counts = df['LinkedIn Status'].value_counts()
df[['name', linkedin_col, 'LinkedIn Status']].head(10), status_counts

(                        name  \
 0                Venkatesh R   
 1                 Mark Jawut   
 2              Avinash Kumar   
 3                Wilson Juma   
 4                     Suneel   
 5  Aishwarya kumar choudhary   
 6            AAKASH JANGEETI   
 7               Arushi Yadav   
 8           Aashish Ailawadi   
 9             Aashish Sharma   
 
                       What is your LinkedIn profile? LinkedIn Status  
 0      https://linkedin.com/in/venkatesh-r-42845a282           Valid  
 1                                          MarkJawut      Incomplete  
 2                  linkedin.com/in/Avinash.kumar5167           Valid  
 3                      linkedin.com/in/wilsonjuma254           Valid  
 4                                                NaN         Missing  
 5  www.linkedin.com/in/ aishwarya-kumar-choudhary...           Valid  
 6     https://linkedin.com/in/aakash-royal-a8015821b           Valid  
 7  https://www.linkedin.com/in/arushi-yadav-68744...      

doing similar for job title 

In [42]:
# Check if the 'Job Title' column is empty or missing
df['Job Title Status'] = df['Job Title'].apply(
    lambda x: 'Missing' if pd.isna(x) or str(x).strip() == '' else 'Present'
)

# Count how many are missing vs present
job_title_status_counts = df['Job Title Status'].value_counts()

# Show a sample of the updated data
df[['name', 'Job Title', 'Job Title Status']].head(10), job_title_status_counts 


#show details of missing job titles
missing_job_titles = df[df['Job Title Status'] == 'Missing'][['name', 'email', 'Job Title']]    
print("Details of missing job titles:")
print(missing_job_titles.head())


Details of missing job titles:
             name            email Job Title
317  Maxwel Maina  *****@gmail.com       NaN
445        Raihan  *****@gmail.com       NaN


Completing Step 1 by saving all the  saving all new data in a new csv file called cleaned_output.csv

In [44]:
# Save to a new xl file
output_file_path = 'Cleaned_Data.xlsx'  
df.to_excel(output_file_path, index=False)

# Saving to a CSV file
output_csv_path = 'cleaned_output.csv '
df.to_csv(output_csv_path, index=False)